In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
!apt install chromium-chromedriver
!pip install selenium

from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (83.0.4103.61-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [7]:
from bs4 import BeautifulSoup
import pandas as pd

genre_dic = {}

def makeUrl(dateList,i, kinds):
  if kinds == 'playstore': 
    url = 'https://www.mobileindex.com/app/get_rank_all?rt=r&mk=2&c=kr&t=app&rs=100&d=' + dateList[i]
  else:
    url = 'https://www.mobileindex.com/app/get_rank_all?rt=r&mk=1&c=kr&t=app&rs=100&d=' + dateList[i]
  return url

def getGenreUrl(url):
  driver = webdriver.Chrome('chromedriver',options=options)
  driver.get(url)
  driver.implicitly_wait(3)

  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  genreURL = soup.select('div.item-info > a')
  return genreURL

def get_genre(url,appname):
  if appname in genre_dic:
    return genre_dic[appname]
  dr = webdriver.Chrome('chromedriver',options=options)
  dr.get(url)
  dr.implicitly_wait(3)
  ht = dr.page_source
  so = BeautifulSoup(ht, 'html.parser')
  genre = so.select('tr.text-center')[0].text
  index = genre.find('2')
  genre_dic[appname] = genre[13:(index-1)]
  return genre[13:(index-1)]

def separateAppRank(rank,isYear):
  free_rank = []
  pay_rank = []
  sales_rank = []
  if isYear == 2019:
    #무료, 유료 순위만 존재
    for j in range(len(rank)):
      free_rank.append(rank[j][1::2])
      pay_rank.append(rank[j][0::2])
    return [free_rank, pay_rank]
  else:
    #무료, 유료, 매출순위 존재
    for j in range(len(rank)):
      free_rank.append(free_rank_input)
      pay_rank.append(rank[j][2::3])
      sales_rank.append(rank[j][0::3])
    return [free_rank, pay_rank, sales_rank]


def getAppRank(url, year):
  driver = webdriver.Chrome('chromedriver',options=options)
  driver.get(url)
  driver.implicitly_wait(3)

  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  notices = soup.select('span.appname')
  
  tmp = []
  for n in notices:
    tmp.append(n.text.strip())
  return tmp

In [8]:
#2019
appRank = []
date_2019 = ['2019-01-01']#,'2019-02-01','2019-03-01','2019-04-01','2019-05-01','2019-06-01','2019-07-01','2019-08-01','2019-09-01','2019-10-01','2019-11-01','2019-12-01']
date_size = len(date_2019)
kinds = ['playstore','appstore']

p_appRank_free_2019 = []
p_appRank_pay_2019 = []
a_appRank_free_2019 = []
a_appRank_pay_2019 = []

for kind in kinds:
  appRank = []
  appGenre = []
  for i in range(date_size):
    url = makeUrl(date_2019,i,kind)
    tmp = getAppRank(url,2019)
    appRank.append(tmp)
    genreURL = getGenreUrl(url)
    genreList = list()
    for j in range(len(genreURL)):
      genreList.append(get_genre('https://www.mobileindex.com/'+genreURL[j]['href'],tmp[j]))
    appGenre.append(genreList)
    print(kind,' data of ',date_2019[i],' is done')

  sep = separateAppRank(appRank,2019)
  print('sep is done')
  sep2 = separateAppRank(appGenre,2019)
  print('sep2 is done')
  if kind == 'playstore': 
    p_appRank_free_2019 = sep[0]
    p_appRank_pay_2019 = sep[1]
    p_appGenre_free_2019 = sep2[0]
    p_appGenre_pay_2019 = sep2[1]
  else:
    a_appRank_free_2019 = sep[0]
    a_appRank_pay_2019 = sep[1]
    a_appGenre_free_2019 = sep2[0]
    a_appGenre_pay_2019 = sep2[1]

WebDriverException: ignored

In [ ]:
#2020
appRank = []
date_2020 = ['2020-01-01'] #,'2020-02-01','2020-03-01','2020-04-01','2020-05-01','2020-06-01']
date_size = len(date_2020)
kinds = ['playstore','appstore']

p_sales_rank = []
a_sales_rank = []
p_appRank_free_2020 = []
p_appRank_pay_2020 = []
a_appRank_free_2020 = []
a_appRank_pay_2020 = []

for kind in kinds:
  appRank = []
  appGenre = []
  for i in range(date_size):
    url = makeUrl(date_2020,i,kind)
    tmp = getAppRank(url,2020)
    appRank.append(tmp)
    genreURL = getGenreUrl(url)
    genreList = list()
    for j in range(len(genreURL)):
      genreList.append(get_genre('https://www.mobileindex.com/'+genreURL[j]['href'],tmp[j]))
    appGenre.append(genreList)
    print(kind,' data of ',data_2019[i],' is done')
  
  sep = separateAppRank(appRank,2020)
  sep2 = separateAppRank(appGenre,2019)
  
  if kind == 'playstore':
    p_appRank_free_2020 = sep[0]
    p_appRank_pay_2020 = sep[1]
    p_sales_rank = sep[2]
    p_appGenre_free_2020 = sep2[0]
    p_appGenre_pay_2020 = sep2[1]
    p_sales_genre = sep2[2]
  else:
    a_appRank_free_2020 = sep[0]
    a_appRank_pay_2020 = sep[1]
    a_sales_rank = sep[2]
    a_appGenre_free_2020 = sep2[0]
    a_appGenre_pay_2020 = sep2[1]
    a_sales_genre = sep2[2]

In [ ]:
p_appRank_free = list()
for i in range(len(p_appRank_free_2019)):
  p_appRank_free.append(p_appRank_free_2019[i])
  p_appRank_free.append(p_appGenre_free_2019[i])
for i in range(len(p_appRank_free_2020)):
  p_appRank_free.append(p_appRank_free_2020[i])
  p_appRank_free.append(p_appGenre_free_2020[i])

a_appRank_free = list()
for i in range(len(a_appRank_free_2019)):
  a_appRank_free.append(a_appRank_free_2019[i])
  a_appRank_free.append(a_appGenre_free_2019[i])
for i in range(len(a_appRank_free_2020)):
  a_appRank_free.append(a_appRank_free_2020[i])
  a_appRank_free.append(a_appGenre_free_2020[i])


In [ ]:
col = []
for i in range(1):
  col.append('19-'+str(i+1))
  col.append('19-'+str(i+1)+'-gen')
for i in range(1):
  col.append('20-'+str(i+1))
  col.append('20-'+str(i+1)+'-gen')

appRank_free_playstore_df = pd.DataFrame(p_appRank_free)
appRank_free_playstore_df = appRank_free_playstore_df.T
appRank_free_playstore_df.columns = col
appRank_free_playstore_df.to_csv('/content/gdrive/My Drive/dacon/appRank_free_playstore.csv', encoding='utf-8-sig')

appRank_free_appstore_df = pd.DataFrame(a_appRank_free)
appRank_free_appstore_df = appRank_free_appstore_df.T
appRank_free_appstore_df.columns = col
appRank_free_appstore_df.to_csv('/content/gdrive/My Drive/dacon/appRank_free_appstore.csv', encoding='utf-8-sig')

In [ ]:
p_appRank_pay = list()
for i in range(len(p_appRank_free_2019)):
  p_appRank_pay.append(p_appRank_free_2019[i])
  p_appRank_pay.append(p_appGenre_free_2019[i])
for i in range(len(p_appRank_free_2020)):
  p_appRank_pay.append(p_appRank_free_2020[i])
  p_appRank_pay.append(p_appGenre_free_2020[i])

a_appRank_pay = list()
for i in range(len(a_appRank_pay_2019)):
  a_appRank_pay.append(a_appRank_pay_2019[i])
  a_appRank_pay.append(a_appGenre_pay_2019[i])
for i in range(len(a_appRank_pay_2020)):
  a_appRank_pay.append(a_appRank_pay_2020[i])
  a_appRank_pay.append(a_appGenre_pay_2020[i])


In [ ]:
appRank_pay_playstore_df = pd.DataFrame(p_appRank_pay)
appRank_pay_playstore_df = appRank_pay_playstore_df.T
appRank_pay_playstore_df.columns = col
appRank_pay_playstore_df.to_csv('/content/gdrive/My Drive/dacon/appRank_pay_playstore.csv', encoding='utf-8-sig')

appRank_pay_appstore_df = pd.DataFrame(a_appRank_pay)
appRank_pay_appstore_df = appRank_pay_appstore_df.T
appRank_pay_appstore_df.columns = col
appRank_pay_appstore_df.to_csv('/content/gdrive/My Drive/dacon/appRank_pay_appstore.csv', encoding='utf-8-sig')

In [ ]:
p_sales_r = list()
for i in range(len(p_sales_rank)):
  p_sales_r.append(p_sales_rank[i])
  p_sales_r.append(p_sales_genre[i])

a_sales_r = list()
for i in range(len(a_sales_rank)):
  a_sales_r.append(a_sales_rank[i])
  a_sales_r.append(a_sales_genre[i])

In [ ]:
p_sales_rank_df = pd.DataFrame(p_sales_r)
p_sales_rank_df = p_sales_rank_df.T
p_sales_rank_df.columns = 
p_sales_rank_df.to_csv('/content/gdrive/My Drive/dacon/app_sales_rank_playstore.csv', encoding='utf-8-sig')

a_sales_rank_df = pd.DataFrame(a_sales_r)
a_sales_rank_df = a_sales_rank_df.T
a_sales_rank_df.columns = ['2020-01-01','2020-02-01','2020-03-01','2020-04-01','2020-05-01','2020-06-01']
a_sales_rank_df.to_csv('/content/gdrive/My Drive/dacon/app_sales_rank_appstore.csv', encoding='utf-8-sig')